In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!rm -rf flowers/
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pacificrm/skindiseasedataset
!unzip SkinDisease.zip

In [ ]:
train_dir = "/SkinDisease/SkinDisease/train"
test_dir = "/SkinDisease/SkinDisease/test"

In [ ]:
classes = [
    "Acne", "Actinic Keratosis", "Benign Tumors", "Bullous", "Candidiasis",
    "Drug Eruption", "Eczema", "Infestations/Bites", "Lichen", "Lupus",
    "Moles", "Psoriasis", "Rosacea", "Seborrheic Keratoses", "Skin Cancer",
    "Sun/Sunlight Damage", "Tinea", "Unknown/Normal", "Vascular Tumors",
    "Vasculitis", "Vitiligo", "Warts"
]

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
print(f"Number of training samples: {train_generator.samples}")
print(f"Number of validation samples: {val_generator.samples}")
print(f"Number of testing samples: {test_generator.samples}")

In [ ]:
def show_images(generator, classes, num_images=8):
    images, labels = next(generator)
    images = images[:num_images]
    labels = labels[:num_images]

    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        axes[i].imshow(images[i])
        axes[i].set_title(classes[np.argmax(labels[i])])
        axes[i].axis('off')
    plt.show()

In [ ]:
show_images(train_generator, classes)

In [ ]:
# def residual_block(x, filters, stride=1, use_dropout=False):
#     shortcut = x
#     x = layers.Conv2D(filters, kernel_size=3, strides=stride, padding='same')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
#     x = layers.BatchNormalization()(x)
#     if use_dropout:
#         x = layers.Dropout(0.5)(x)
#     if shortcut.shape[-1] != filters or stride != 1:
#         shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
#     x = layers.add([x, shortcut])
#     x = layers.ReLU()(x)
#     return x

In [ ]:
# def build_resnet(input_shape=(224, 224, 3), num_classes=22, use_dropout=False):
#     inputs = layers.Input(shape=input_shape)
#     x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
#     x = residual_block(x, 64, stride=1, use_dropout=use_dropout)
#     x = residual_block(x, 128, stride=2, use_dropout=use_dropout)
#     x = residual_block(x, 256, stride=2, use_dropout=use_dropout)
#     x = residual_block(x, 512, stride=2, use_dropout=use_dropout)
#     x = layers.GlobalAveragePooling2D()(x)
#     if use_dropout:
#         x = layers.Dropout(0.5)(x)
#     outputs = layers.Dense(num_classes, activation='softmax')(x)
#     model = models.Model(inputs, outputs)
#     return model

In [ ]:
# model = build_resnet(num_classes=22, use_dropout=True)
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    baseline=0.90
)

callbacks = [reduce_lr, checkpoint, early_stopping]

In [ ]:
model = models.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and Validation Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("resnet_model.h5")
print("Model saved as resnet_model.h5")

In [ ]:
model = tf.keras.models.load_model("best_model.h5")
print("Model loaded successfully")

In [ ]:
def predict_and_show(model, generator, class_names, num_images=5):
    images, labels = next(generator)
    predictions = model.predict(images[:num_images])
    for i in range(num_images):
        plt.imshow(images[i])
        plt.title(f"Predicted: {class_names[np.argmax(predictions[i])]}\nTrue: {class_names[np.argmax(labels[i])]}" )
        plt.axis('off')
        plt.show()

predict_and_show(model, test_generator, classes, num_images=10)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Accuracy on the test set: {test_accuracy * 100:.2f}%")

In [ ]:
test_generator.reset()
preds = model.predict(test_generator, verbose=0)
preds = np.argmax(preds, axis=1)

cm = confusion_matrix(test_generator.classes, preds)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(
    y_true=test_generator.classes,
    y_pred=preds,
    target_names=classes,
    digits=4
))